In [ ]:
import numpy as np

In [ ]:
mesh_name = "ifa_meander_test.msh"
save_mesh_folder = 'data/gmsh_files/'
ifa_meander_mat = 'data/antennas_mesh/ifa_meander_test.mat'
ifa_meander_msh = save_mesh_folder + mesh_name

In [ ]:
fC      = 868e6
fLow    = 820e6
fHigh   = 940e6
nPoints = 88

In [ ]:
# Terminal size 
t_a = 49.22 / 1000   # Hauteur
t_b = 13.51 / 1000  # Largeur
x_t = np.array([-t_a, 0, 0, -t_a])
y_t = np.array([t_b/2, t_b/2, -t_b/2, -t_b/2])